In [1]:
from __future__ import print_function

import datetime
from tensorflow.python import keras
from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras import backend as K

In [2]:
now = datetime.datetime.now

In [3]:
batch_size = 128
num_classes = 5
epochs = 5

In [4]:
# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

In [5]:
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [6]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [7]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [8]:
# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [9]:
# create complete model
model = Sequential(feature_layers + classification_layers)

In [10]:
# train model for 5-digit classification [0..4]
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================]30596/30596 [==============================] - 6s 211us/step - loss: 0.2096 - acc: 0.9336 - val_loss: 0.0522 - val_acc: 0.9840

Epoch 2/5
30596/30596 [==============================]30596/30596 [==============================] - 4s 128us/step - loss: 0.0709 - acc: 0.9779 - val_loss: 0.0258 - val_acc: 0.9910

Epoch 3/5
30596/30596 [==============================]30596/30596 [==============================] - 4s 128us/step - loss: 0.0494 - acc: 0.9845 - val_loss: 0.0185 - val_acc: 0.9936

Epoch 4/5
30596/30596 [==============================]30596/30596 [==============================] - 4s 132us/step - loss: 0.0381 - acc: 0.9889 - val_loss: 0.0149 - val_acc: 0.9947

Epoch

In [11]:
# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False

In [12]:
# transfer: train dense layers for new classification task [5..9]
train_model(model,
            (x_train_gte5, y_train_gte5),
(x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================]29404/29404 [==============================] - 2s 84us/step - loss: 0.3909 - acc: 0.8824 - val_loss: 0.0976 - val_acc: 0.9704

Epoch 2/5
29404/29404 [==============================]29404/29404 [==============================] - 2s 81us/step - loss: 0.1288 - acc: 0.9604 - val_loss: 0.0616 - val_acc: 0.9798

Epoch 3/5
29404/29404 [==============================]29404/29404 [==============================] - 2s 81us/step - loss: 0.0938 - acc: 0.9715 - val_loss: 0.0479 - val_acc: 0.9842

Epoch 4/5
29404/29404 [==============================]29404/29404 [==============================] - 2s 76us/step - loss: 0.0787 - acc: 0.9763 - val_loss: 0.0411 - val_acc: 0.9868

Epoch 5/5
29404/29404 [==============================]29404/29404 [==============================] - 2s 81us/step - loss: 0.0688 - acc: 0.9786 - val_loss: 